##### Open Source convert .dlis to .las v2.0 using dlisio and lasio python packages

## Using DLISIO v0.3.5

## Still a work in progress.  Need to dos:
 - add functionality to manipulate the units of the index track because often it is in 0.1 in.
 
Work released under MIT License (MIT)

## But this code will handle multiple embedded files within a dlis and multiple frames in a single dlis.  This code will also handle scenarios where values are actually lists of values.

In [16]:
%run -i "func_dlis_to_las.py"

import os
import pandas as pd
import numpy as np
import lasio
import dlisio
from datetime import datetime
import re

## Fill in the file path for the dlis you want to read in the below cell:

In [17]:
filepath = r"C:\Appl\WPy64-3830\notebooks\DLISIO_Notebooks\WL_RAW_CAL-DEN-GR-NEU-REMP_MWD_1.DLIS"

## Fill in the file path for where you want the output las file to go in the below cell:

In [20]:
output_folder_location = "C:\\Appl\\WPy64-3830\\notebooks\\DLISIO_Notebooks"

## Call the function (available in the same repo)

In [21]:
convert_dlis_to_las(filepath, output_folder_location, null=-999.25)

-------------
Physical File
-------------
Number of Logical Files : 1

Description : LogicalFile(Run_4)
Frames      : 2
Channels    : 282


Processing TDEP
Completed TDEP
Processing 6TIM
Completed 6TIM
Processing ROP5_RM
Completed ROP5_RM
Processing RFSA
Completed RFSA
Processing RFSB
Completed RFSB
Processing RFSL
Completed RFSL
Processing RFSU
Completed RFSU
Processing RFSR
Completed RFSR
Processing QC_SIGM
Completed QC_SIGM
Processing QC_SPEC
Completed QC_SPEC
Processing CHY_WALK2
Completed CHY_WALK2
Processing CSI_WALK2
Completed CSI_WALK2
Processing CCA_WALK2
Completed CCA_WALK2
Processing CCHL_WALK2
Completed CCHL_WALK2
Processing CSUU_WALK2
Completed CSUU_WALK2
Processing CGD_WALK2
Completed CGD_WALK2
Processing CTI_WALK2
Completed CTI_WALK2
Processing CNC_WALK2
Completed CNC_WALK2
Processing DWSI_WALK2
Completed DWSI_WALK2
Processing DWCA_WALK2
Completed DWCA_WALK2
Processing DWFE_WALK2
Completed DWFE_WALK2
Processing DWSU_WALK2
Completed DWSU_WALK2
Processing DWTI_WALK2
Comple

Completed TNPL_UNC
Processing TNPU_UNC
Completed TNPU_UNC
Processing TNPR_UNC
Completed TNPR_UNC
Processing BPHI_UNC
Completed BPHI_UNC
Processing BPHB_UNC
Completed BPHB_UNC
Processing BPHL_UNC
Completed BPHL_UNC
Processing BPHU_UNC
Completed BPHU_UNC
Processing BPHR_UNC
Completed BPHR_UNC
Processing DBPHI
Completed DBPHI
Processing DBPHB
Completed DBPHB
Processing DBPHL
Completed DBPHL
Processing DBPHU
Completed DBPHU
Processing DBPHR
Completed DBPHR
Processing QC_BPHI
Completed QC_BPHI
Processing QC_TNPH
Completed QC_TNPH
Processing TAB_NEU
Completed TAB_NEU
Processing MONT
Completed MONT
If these are different lengths, something is wrong:
458
458
458
Writing: [C:\Appl\WPy64-3830\notebooks\DLISIO_Notebooks\WL_RAW_CAL-DEN-GR-NEU-REMP_MWD_1_converted_with_python_1.las]

Processing TDEP
Completed TDEP
Processing 1TIM
Completed 1TIM
Processing SAZ1_P1F
Completed SAZ1_P1F
Processing SAZ2_P1F
Completed SAZ2_P1F
Processing LSAZ_P1F
Completed LSAZ_P1F
Processing PESI
Completed PESI
Processi